1. В коде из методички реализуйте один или несколько критериев останова: количество листьев, количество используемых признаков, глубина дерева и т. д.

In [1]:
import numpy as np
from sklearn import datasets
from sklearn import model_selection

In [34]:
class Node:

    def __init__(self, index, t, true_branch, false_branch):
        self.index = index
        self.t = t
        self.true_branch = true_branch
        self.false_branch = false_branch

class Leaf:

    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()


    def predict(self):
        classes = {}
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        prediction = max(classes, key=classes.get)
        return prediction
    

def gini(labels):
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p**2
    return impurity


def quality(left_labels, right_labels, current_gini):
    p = float(left_labels.shape[0] / (left_labels.shape[0] + right_labels.shape[0]))
    return current_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)


def split(data, labels, index, t):
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
    return true_data, false_data, true_labels, false_labels


def find_best_split(data, labels, min_leaf):
    current_gini = gini(labels)
    best_quality = 0
    best_t = None
    best_index = None
    n_features = data.shape[1]
    for index in range(n_features):
        t_values = np.unique([row[index] for row in data])
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            current_quality = quality(true_labels, false_labels, current_gini)
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index
    return best_quality, best_t, best_index


def build_tree(data, labels, min_leaf=5, max_depth=3):
    quality, t, index = find_best_split(data, labels, min_leaf)
    if quality == 0 or max_depth < 1:
        return Leaf(data, labels)
    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
    max_depth -= 1
    true_branch = build_tree(true_data, true_labels, min_leaf, max_depth)
    false_branch = build_tree(false_data, false_labels, min_leaf, max_depth)
    return Node(index, t, true_branch, false_branch)


def classify_object(obj, node):
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer
    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)
    

def predict(data, tree):
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [3]:
def print_tree(node, spacing=""):

    # Если лист, то выводим его прогноз
    if isinstance(node, Leaf):
        print(spacing + "Прогноз:", node.prediction)
        return

    # Выведем значение индекса и порога на этом узле
    print(spacing + 'Индекс', str(node.index))
    print(spacing + 'Порог', str(node.t))

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [4]:
# сгенерируем данные
classification_data, classification_labels = datasets.make_classification(n_features = 2, n_informative = 2, 
                                                      n_classes = 2, n_redundant=0, 
                                                      n_clusters_per_class=1, random_state=5)

In [5]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(classification_data, classification_labels, test_size = 0.3, random_state = 1)

In [39]:
my_tree = build_tree(train_data, train_labels, max_depth=3)

In [40]:
print_tree(my_tree)

Индекс 0
Порог 0.16261402870113306
--> True:
  Индекс 1
  Порог -1.5208896621663803
  --> True:
    Индекс 0
    Порог -0.9478301462477035
    --> True:
      Прогноз: 0
    --> False:
      Прогноз: 1
  --> False:
    Прогноз: 0
--> False:
  Прогноз: 1


2. Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.
Комментарий:
Критерий останова "глубина дерева" я реализовал не в коде из методички, а уже в измененном коде для задачи регрессии. На суть это не влияет, поскольку функция build_tree не зависит от особенностей решаемой задачи (она просто строит дерево и ничего не знает о контексте).

In [23]:
class Node:

    def __init__(self, index, t, true_branch, false_branch):
        self.index = index
        self.t = t
        self.true_branch = true_branch
        self.false_branch = false_branch

class Leaf:

    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()


    def predict(self):
        return self.labels.mean()
    

def mse(labels):
    return np.mean((labels - labels.mean())**2)


def quality(left_labels, right_labels, root_criterion, criterion):
    p = float(left_labels.shape[0] / (left_labels.shape[0] + right_labels.shape[0]))
    return root_criterion - p * criterion(left_labels) - (1 - p) * criterion(right_labels)


def split(data, labels, index, t):
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
    return true_data, false_data, true_labels, false_labels


def find_best_split(data, labels, min_leaf):
    root_mse = mse(labels)
    best_quality = 0
    best_t = None
    best_index = None
    n_features = data.shape[1]
    for index in range(n_features):
        t_values = np.unique([row[index] for row in data])
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            current_quality = quality(true_labels, false_labels, root_mse, mse)
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index
    return best_quality, best_t, best_index


def build_tree(data, labels, min_leaf=5):
    quality, t, index = find_best_split(data, labels, min_leaf)
    if quality == 0:
        return Leaf(data, labels)
    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
    true_branch = build_tree(true_data, true_labels, min_leaf)
    false_branch = build_tree(false_data, false_labels, min_leaf)
    return Node(index, t, true_branch, false_branch)


def classify_object(obj, node):
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer
    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)
    

def predict(data, tree):
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [20]:
data, y = datasets.make_regression(n_features=2, n_informative=2, random_state=5)

In [21]:
train_data_regr, test_data_regr, train_y_regr, test_y_regr = model_selection.train_test_split(data, y, test_size=0.3, random_state=1)

In [24]:
my_tree = build_tree(train_data_regr, train_y_regr)

In [25]:
print_tree(my_tree)

Индекс 0
Порог -0.10061434630710828
--> True:
  Индекс 0
  Порог -0.8568531547160899
  --> True:
    Прогноз: -109.75655471490919
  --> False:
    Индекс 0
    Порог -0.5732155560138283
    --> True:
      Прогноз: -54.35634172577482
    --> False:
      Индекс 1
      Порог -0.3058530211666308
      --> True:
        Прогноз: -29.105630694331246
      --> False:
        Прогноз: -10.772916465924025
--> False:
  Индекс 0
  Порог 0.9068894675659355
  --> True:
    Индекс 1
    Порог 0.6566194702604272
    --> True:
      Индекс 1
      Порог -1.0650326193820066
      --> True:
        Прогноз: 7.798014762375311
      --> False:
        Индекс 0
        Порог 0.41367880834311616
        --> True:
          Прогноз: 17.019366109004093
        --> False:
          Прогноз: 35.95087900163848
    --> False:
      Индекс 0
      Порог 0.34691932708774675
      --> True:
        Прогноз: 37.4238776327042
      --> False:
        Прогноз: 61.9558421220885
  --> False:
    Индекс 0
    Порог 1.3